## UTMCC DataViz Module 20 Final Team Project
### Food Deserts in the Austin, Texas Metro Area

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pandas as pd
from scipy import stats
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
# import datetime as dt
# import calendar
# import random

# %matplotlib inline
# from matplotlib import style
# style.use('fivethirtyeight')
# import matplotlib.pyplot as plt
# import statistics
# from flask import Flask, jsonify
# from mpl_toolkits.mplot3d import Axes3D

# # Python SQL toolkit and Object Relational Mapper
# import sqlite3
# import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func
# from sqlalchemy import extract

In [3]:
# Import and read csv files
food_atlas_df = pd.read_csv("resources/food_desert_austin_censustract.csv")
# food_var_df = pd.read_csv("resources/food_access_variable_lookup.csv")

In [4]:
food_atlas_df.head()

,Unnamed: 0,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,61529,48021950100,Texas,Bastrop,0,8608,3063,0,0,0.000000,...,1028,6507,383,53,4,65,1596,2660,51,351
1,61530,48021950200,Texas,Bastrop,1,7955,2625,0,186,0.023381,...,891,4521,1328,32,14,76,1984,3674,148,444
2,61531,48021950300,Texas,Bastrop,0,12927,4734,0,71,0.005492,...,1468,9971,1213,156,18,88,1481,3012,95,568
3,61532,48021950400,Texas,Bastrop,1,7984,3127,0,456,0.057114,...,1199,6312,800,78,3,62,729,1711,255,261
4,61533,48021950501,Texas,Bastrop,0,8008,2168,0,1519,0.189685,...,566,5209,849,37,12,123,1778,3253,128,285


In [5]:
# food_var_df.head()

## Preprocessing the data

In [6]:
food_atlas_df.dtypes

Unnamed: 0        int64
CensusTract       int64
State            object
County           object
Urban             int64
                  ...  
TractAIAN         int64
TractOMultir      int64
TractHispanic     int64
TractHUNV         int64
TractSNAP         int64
Length: 148, dtype: object

In [7]:
# Create new df keeping only Texas
food_texas_df = food_atlas_df[(food_atlas_df["State"]=="Texas")]
food_texas_df

,Unnamed: 0,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,61529,48021950100,Texas,Bastrop,0,8608,3063,0,0,0.000000,...,1028,6507,383,53,4,65,1596,2660,51,351
1,61530,48021950200,Texas,Bastrop,1,7955,2625,0,186,0.023381,...,891,4521,1328,32,14,76,1984,3674,148,444
2,61531,48021950300,Texas,Bastrop,0,12927,4734,0,71,0.005492,...,1468,9971,1213,156,18,88,1481,3012,95,568
3,61532,48021950400,Texas,Bastrop,1,7984,3127,0,456,0.057114,...,1199,6312,800,78,3,62,729,1711,255,261
4,61533,48021950501,Texas,Bastrop,0,8008,2168,0,1519,0.189685,...,566,5209,849,37,12,123,1778,3253,128,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,66673,48491021507,Texas,Williamson,1,9196,3293,0,1,0.000109,...,444,6315,1227,421,15,44,1174,2710,179,277
346,66674,48491021508,Texas,Williamson,1,6531,2402,0,0,0.000000,...,301,4744,761,303,7,28,688,1627,23,133
347,66675,48491021601,Texas,Williamson,0,3278,1120,0,0,0.000000,...,236,2569,102,19,6,15,567,954,5,26
348,66676,48491021602,Texas,Williamson,0,2857,959,0,0,0.000000,...,332,2464,47,19,0,16,311,906,12,106


In [8]:
# Create new df keeping only select Counties in the Austin Metro Area
counties = ["Bastrop", "Caldwell", "Hays", "Travis", "Williamson"]
food_austin_df = food_texas_df.loc[food_texas_df["County"].isin(counties)]
food_austin_df

,Unnamed: 0,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,61529,48021950100,Texas,Bastrop,0,8608,3063,0,0,0.000000,...,1028,6507,383,53,4,65,1596,2660,51,351
1,61530,48021950200,Texas,Bastrop,1,7955,2625,0,186,0.023381,...,891,4521,1328,32,14,76,1984,3674,148,444
2,61531,48021950300,Texas,Bastrop,0,12927,4734,0,71,0.005492,...,1468,9971,1213,156,18,88,1481,3012,95,568
3,61532,48021950400,Texas,Bastrop,1,7984,3127,0,456,0.057114,...,1199,6312,800,78,3,62,729,1711,255,261
4,61533,48021950501,Texas,Bastrop,0,8008,2168,0,1519,0.189685,...,566,5209,849,37,12,123,1778,3253,128,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,66673,48491021507,Texas,Williamson,1,9196,3293,0,1,0.000109,...,444,6315,1227,421,15,44,1174,2710,179,277
346,66674,48491021508,Texas,Williamson,1,6531,2402,0,0,0.000000,...,301,4744,761,303,7,28,688,1627,23,133
347,66675,48491021601,Texas,Williamson,0,3278,1120,0,0,0.000000,...,236,2569,102,19,6,15,567,954,5,26
348,66676,48491021602,Texas,Williamson,0,2857,959,0,0,0.000000,...,332,2464,47,19,0,16,311,906,12,106


In [9]:
#begin income column creation (target variable)
food_austin_df['MedianFamilyIncome']

0      70516
1      67792
2      75462
3      62375
4      65079
       ...  
345    66820
346    83241
347    57389
348    63125
349    58902
Name: MedianFamilyIncome, Length: 350, dtype: int64

In [10]:
conditions = [(food_austin_df['MedianFamilyIncome'] <= 15000), 
              (food_austin_df['MedianFamilyIncome'] > 15000) & (food_austin_df['MedianFamilyIncome'] < 75000),
              (food_austin_df['MedianFamilyIncome'] >= 75000)]
values = ['low', 'middle', 'high']
food_austin_df["Income"] = np.select(conditions, values)
food_austin_df.head()

,Unnamed: 0,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,...,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,Income
0,61529,48021950100,Texas,Bastrop,0,8608,3063,0,0,0.000000,...,6507,383,53,4,65,1596,2660,51,351,middle
1,61530,48021950200,Texas,Bastrop,1,7955,2625,0,186,0.023381,...,4521,1328,32,14,76,1984,3674,148,444,middle
2,61531,48021950300,Texas,Bastrop,0,12927,4734,0,71,0.005492,...,9971,1213,156,18,88,1481,3012,95,568,high
3,61532,48021950400,Texas,Bastrop,1,7984,3127,0,456,0.057114,...,6312,800,78,3,62,729,1711,255,261,middle
4,61533,48021950501,Texas,Bastrop,0,8008,2168,0,1519,0.189685,...,5209,849,37,12,123,1778,3253,128,285,middle


In [11]:
# Create new df with select feature columns
food_desert_df = food_austin_df[["LILATracts_1And10", "lasnaphalf", "lasnaphalfshare", "lahunvhalf", "lahunvhalfshare", "lasnap1", "lasnap1share", "lahunv1", "lahunv1share", "lasnap10", "lasnap10share", "lahunv10", "lahunv10share", "lasnap20", "lasnap20share",  "lahunv20", "lahunv20share"]]
food_desert_df

,LILATracts_1And10,lasnaphalf,lasnaphalfshare,lahunvhalf,lahunvhalfshare,lasnap1,lasnap1share,lahunv1,lahunv1share,lasnap10,lasnap10share,lahunv10,lahunv10share,lasnap20,lasnap20share,lahunv20,lahunv20share
0,0,325.133846,0.106149,48.548857,0.015850,319.907937,0.104443,47.889963,0.015635,5.159378,0.001684,0.922672,0.000301,0.0,0.0,0.0,0.0
1,1,368.584869,0.140413,120.671667,0.045970,233.501673,0.088953,81.658810,0.031108,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0,590.025451,0.124636,100.788957,0.021290,549.545092,0.116085,84.946165,0.017944,48.577263,0.010261,8.752010,0.001849,0.0,0.0,0.0,0.0
3,0,241.111748,0.077106,191.810180,0.061340,188.984455,0.060436,114.453080,0.036602,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0,307.709167,0.141932,144.460704,0.066633,307.703411,0.141930,144.458152,0.066632,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,0,253.269297,0.076911,147.346423,0.044745,200.157469,0.060783,94.092539,0.028574,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
346,0,65.461476,0.027253,3.168106,0.001319,37.486129,0.015606,0.346386,0.000144,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
347,1,24.931507,0.022260,6.618853,0.005910,24.931507,0.022260,6.618853,0.005910,15.243362,0.013610,2.243933,0.002004,0.0,0.0,0.0,0.0
348,0,104.153688,0.108607,13.420352,0.013994,104.153688,0.108607,13.420352,0.013994,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


## SQLite database

In [12]:
# # SQLAlchemy Create Engine, to access and query the SQLite database file
# engine = create_engine("sqlite:///food_desert.sqlite", echo=True)
# sqlite_connection = engine.connect()

In [13]:
# to reflect the database into a new model
#Base = automap_base()
# to reflect the db tables 
#Base.prepare(engine, reflect=True)

### Create the database

In [14]:
# # Create the database
# conn = sqlite3.connect("DB1.sqlite")
# cur = conn.cursor()

In [15]:
# cur.execute('CREATE TABLE Food_desert_Austin (LILATracts_1And10, lasnaphalf, lasnaphalfshare, lahunvhalf, lahunvhalfshare, lasnap1, lasnap1share, lahunv1, lahunv1share, lasnap10, lasnap10share, lahunv10, lahunv10share, lasnap20, lasnap20share,  lahunv20, lahunv20share)')
# conn.commit()

In [16]:
# # Create our session (link) from Python to the DB
# session = Session(engine)

In [17]:
# # Data from dataframe to SQLite
# food_desert_df.to_sql('Food_desert_Austin', conn, if_exists='replace')

In [18]:
# cur.execute('''SELECT * FROM Food_desert_Austin''')

In [19]:
# for row in cur.fetchall():
#     print(row)

In [20]:
# From SQLite to DataFrame
# food_desert2_df = pd.DataFrame(cur.fetchall(), columns=['LILATracts_1And10', 'lasnaphalf', 'lasnaphalfshare', 'lahunvhalf', 'lahunvhalfshare', 'lasnap1', 'lasnap1share', 'lahunv1', 'lahunv1share', 'lasnap10', 'lasnap10share', 'lahunv10', 'lahunv10share', 'lasnap20', 'lasnap20share',  'lahunv20', 'lahunv20share'])
# print(food_desert2_df)

In [21]:
# # Creating a new dataframe from the SQLite database table.
# food_desert_Austin_df = pd.read_sql_query("SELECT * FROM Food_desert_Austin", conn)

In [22]:
# food_desert_Austin_df

In [23]:
# # Close the database connection
# sqlite_connection.close()

## Machine Learning Model

In [24]:
# Split the preprocessed dataframe from the SQLite database into our features and target arrays
#  Remove LILATracts_1And10 target from features data
y = food_austin_df.Income
X = food_desert_df
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [25]:
y.describe()

count        350
unique         3
top       middle
freq         180
Name: Income, dtype: object

In [26]:
# Scale X
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# #Scale Y
# # Create a StandardScaler instances
# scaler = MinMaxScaler()
# y_train_scaled = scaler.fit_transform(y_train)
# y_test_scaled = scaler.fit_transform(y_test)

# # x = clustered_df[["TotalCoinSupply", "TotalCoinsMined"]]
# # scaler = MinMaxScaler()
# # X_scaled = scaler.fit_transform(x)
# # X_scaled

In [28]:
# Define the model - deep neural net. the number of input features and the hidden nodes for each layer.
# A good rule of thumb for a basic neural network is to have two to three times the amount of neurons in 
# the hidden layer as the number of inputs.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                540       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 861
Trainable params: 861
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile and train the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50


UnimplementedError:  Cast string to float is not supported
	 [[node binary_crossentropy/Cast (defined at <ipython-input-29-affe25a7a652>:3) ]] [Op:__inference_train_function_725]

Function call stack:
train_function


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Create a callback to save the model's weights, and
###   Save and export the results to an HDF5 file, Hierarchical Data Format

In [ ]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Export our model to HDF5 file
nn.save("food_desert_austin.h5")

## Comparisons to other ML models

In [ ]:
# Create a Random Forest classifier, 64 estimators.
rf_model = RandomForestClassifier(n_estimators=64, random_state=78)
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
# Logistic Regression Model Accuracy
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)
log_classifier.fit(X_train,y_train)
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

## Other, Plotting and Charting 

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
plotScatterMatrix(food_desert_df, 20, 10)

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = food_desert_df
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [ ]:
plotCorrelationMatrix(food_desert_df, 24)